In [1]:
import pandas as pd

После внесения изменений в файл models.py необходимо запустить <br>
python manage.py makemigration && python manage.py migrate

In [11]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'hackathon.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [12]:
from database.models import GiftedChild

In [13]:
#Смотрим все поля базы
GiftedChild.__dict__

mappingproxy({'__module__': 'database.models',
              'CLASS': 'C',
              'ART': 'A',
              'SINGER': 'S',
              'DEERS': 'D',
              'PROFESSION_CHOICES': [('C', 'Class'),
               ('A', 'Art'),
               ('S', 'Singer'),
               ('D', 'Deers')],
              '__doc__': 'GiftedChild(uuid, specialization, name, surname, bio, created_at, updated_at)',
              '_meta': <Options for GiftedChild>,
              'DoesNotExist': database.models.GiftedChild.DoesNotExist,
              'MultipleObjectsReturned': database.models.GiftedChild.MultipleObjectsReturned,
              'uuid': <django.db.models.query_utils.DeferredAttribute at 0x7f1ddb9e04f0>,
              'specialization': <django.db.models.query_utils.DeferredAttribute at 0x7f1ddb9e0550>,
              'get_specialization_display': functools.partialmethod(<function Model._get_FIELD_display at 0x7f1ddbe7a790>, , field=<django.db.models.fields.CharField: specialization>),

In [14]:
#Создаем новую запись в БД
#После создания .save() делать не нужно!
child1 = GiftedChild.objects.create(specialization='A', 
                                    name='Певец', 
                                    surname='Певцов', 
                                    bio='Певец Певцов родился в чуме и первое что он сделал - исполнил 7ую симфонию Чайковского. Этот талант не пропадет!')

In [15]:
#проверяем, что всё создалось
child1.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f1ddb8c0b80>,
 'uuid': None,
 'specialization': 'A',
 'name': 'Певец',
 'surname': 'Певцов',
 'bio': 'Певец Певцов родился в чуме и первое что он сделал - исполнил 7ую симфонию Чайковского. Этот талант не пропадет!',
 'created_at': datetime.datetime(2021, 10, 9, 8, 59, 52, 429061, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2021, 10, 9, 8, 59, 52, 429094, tzinfo=<UTC>)}

In [16]:
#в БД так же появился новый объект
GiftedChild.objects.all()

<QuerySet [<GiftedChild: GiftedChild object (1)>]>

In [17]:
#аттрибуты объекта можно кверить по ключам
GiftedChild.objects.first().name

'Певец'

In [18]:
#Ищем в БД по значению поля
child_query = GiftedChild.objects.get(name='Певец')

In [19]:
child_query

<GiftedChild: GiftedChild object (1)>

In [20]:
#Меняем существующие поля
child_query.surname = 'Певищев'
child_query.save()

In [21]:
#Заново кверим базу
child_query_new = GiftedChild.objects.get(name='Певец')

In [22]:
#Проверяем, что фамилия изменилась
child_query_new.surname

'Певищев'

In [23]:
#При этом это один и тот же ребенок
child_query_new == child_query

True

In [24]:
#Создаем второго ребенка
child2 = GiftedChild.objects.create(specialization='D', 
                                    name='Оленевод', 
                                    surname='Оленеводов', 
                                    bio='Оленевод Оленеводов родился в чуме и первое что он сделал - выпас сразу 20 оленей! Этот талант не пропадет!')

In [25]:
#Через фильтр можно делать тонкую настройку выгрузки, выгружаем всех детей с аттрибутом D в поле specialization
GiftedChild.objects.filter(specialization__icontains='D').all()

<QuerySet [<GiftedChild: GiftedChild object (2)>]>

In [26]:
#Забираем первого ребенка из БД
olenevod = GiftedChild.objects.filter(specialization__icontains='D').all()[0]

In [27]:
#чек био
olenevod.bio

'Оленевод Оленеводов родился в чуме и первое что он сделал - выпас сразу 20 оленей! Этот талант не пропадет!'

In [28]:
#добавляем певца в подписчиков оленевода
olenevod.subs.add(child_query_new)

In [29]:
#Так можно достать всех сабов
for i in olenevod.subs.all().values():
    print(i)

{'uuid': 1, 'specialization': 'A', 'name': 'Певец', 'surname': 'Певищев', 'bio': 'Певец Певцов родился в чуме и первое что он сделал - исполнил 7ую симфонию Чайковского. Этот талант не пропадет!', 'created_at': datetime.datetime(2021, 10, 9, 8, 59, 52, 429061, tzinfo=<UTC>), 'updated_at': datetime.datetime(2021, 10, 9, 8, 59, 53, 618542, tzinfo=<UTC>)}


Сабов можно кверить так же, как бдху!

In [10]:
#Фулл зачистка базы, аккуратно, блять!
GiftedChild.objects.all().delete()

(4, {'database.GiftedChild_subscribers': 1, 'database.GiftedChild': 3})

In [35]:
[i for i in GiftedChild.objects.all().values()]

[{'uuid': 1,
  'specialization': 'A',
  'name': 'Певец',
  'surname': 'Певищев',
  'bio': 'Певец Певцов родился в чуме и первое что он сделал - исполнил 7ую симфонию Чайковского. Этот талант не пропадет!',
  'created_at': datetime.datetime(2021, 10, 9, 8, 59, 52, 429061, tzinfo=<UTC>),
  'updated_at': datetime.datetime(2021, 10, 9, 8, 59, 53, 618542, tzinfo=<UTC>)},
 {'uuid': 2,
  'specialization': 'D',
  'name': 'Оленевод',
  'surname': 'Оленеводов',
  'bio': 'Оленевод Оленеводов родился в чуме и первое что он сделал - выпас сразу 20 оленей! Этот талант не пропадет!',
  'created_at': datetime.datetime(2021, 10, 9, 8, 59, 54, 590691, tzinfo=<UTC>),
  'updated_at': datetime.datetime(2021, 10, 9, 8, 59, 54, 590731, tzinfo=<UTC>)}]